<a href="https://colab.research.google.com/github/Granero0011/DS-Unit-2-Sprint-3-Classification-Validation/blob/master/LS_DS_231_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - Logistic Regression

Logistic regression is the baseline for classification models, as well as a handy way to predict probabilities (since those too live in the unit interval). While relatively simple, it is also the foundation for more sophisticated classification techniques such as neural networks (many of which can effectively be thought of as networks of logistic models).

## Lecture - Where Linear goes Wrong
### Return of the Titanic 🚢

You've likely already explored the rich dataset that is the Titanic - let's use regression and try to predict survival with it. The data is [available from Kaggle](https://www.kaggle.com/c/titanic/data), so we'll also play a bit with [the Kaggle API](https://github.com/Kaggle/kaggle-api).

### Get data, option 1: Kaggle API

#### Sign up for Kaggle and get an API token
1. [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. 
2. [Follow these instructions](https://github.com/Kaggle/kaggle-api#api-credentials) to create a Kaggle “API Token” and download your `kaggle.json` file. If you are using Anaconda, put the file in the directory specified in the instructions.

_This will enable you to download data directly from Kaggle. If you run into problems, don’t worry — I’ll give you an easy alternative way to download today’s data, so you can still follow along with the lecture hands-on. And then we’ll help you through the Kaggle process after the lecture._

#### Put `kaggle.json` in the correct location

- ***If you're using Anaconda,*** put the file in the directory specified in the [instructions](https://github.com/Kaggle/kaggle-api#api-credentials).

- ***If you're using Google Colab,*** upload the file to your Google Drive, and run this cell:

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/


#### Install the Kaggle API package and use it to get the data

You also have to join the Titanic competition to have access to the data

In [2]:
!pip install kaggle

In [3]:
!kaggle competitions download -c titanic

403 - Forbidden


### Get data, option 2: Download from the competition page
1. [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. 
2. [Go to the Titanic competition page](https://www.kaggle.com/c/titanic) to download the [data](https://www.kaggle.com/c/titanic/data).

### Get data, option 3: Use Seaborn

```
import seaborn as sns
train = sns.load_dataset('titanic')
```

But Seaborn's version of the Titanic dataset is not identical to Kaggle's version, as we'll see during this lesson!

### Read data

In [6]:
import pandas as pd
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')
train.shape, test.shape

FileNotFoundError: ignored

In [0]:
train.sample(n=5)

In [0]:
test.sample(n=5)

In [0]:
target = 'Survived'
train[target].value_counts(normalize=True)

In [0]:
train.describe(include='number')

In [0]:
train.describe(exclude='number')

### How would we try to do this with linear regression?

https://scikit-learn.org/stable/modules/impute.html

In [0]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

features = ['Pclass', 'Age', 'Fare']
target  = 'Survived'
X_train = train[features]
y_train = train[target]
X_test  = test[features]

imputer = SimpleImputer()
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed  = imputer.transform(X_test)

lin_reg = LinearRegression()
lin_reg.fit(X_train_imputed, y_train)

In [0]:
X_train.shape, X_train_imputed.shape, X_test.shape, X_test_imputed.shape

In [0]:
X_train.head(6)

In [0]:
X_train_imputed[:6]

In [0]:
X_train['Age'].mean()

In [0]:
X_test.tail()

In [0]:
X_test_imputed[-5:]

In [0]:
X_test['Age'].mean()

In [0]:
pd.concat([X_train, X_test])['Age'].mean()

In [0]:
import numpy as np
test_case = np.array([[1, 5, 500]]) # Rich 5-year old in first class
lin_reg.predict(test_case)

In [0]:
y_pred = lin_reg.predict(X_test_imputed)

In [0]:
pd.Series(y_pred).describe()

In [0]:
pd.Series(lin_reg.coef_, X_train.columns)

In [0]:
lin_reg.intercept_

### How would we do this with Logistic Regression?

In [0]:
from sklearn.linear_model import LogisticRegression

# Linear Regression
# lin_reg = LinearRegression()
# lin_reg.fit(X_train_imputed, y_train)
# lin_reg.predict(test_case)

log_reg = LogisticRegression(solver='lbfgs')
log_reg.fit(X_train_imputed, y_train)
print('Prediction for rich 5 year old:', log_reg.predict(test_case))
print('Predicted probabilities for rich 5 year old:', log_reg.predict_proba(test_case))

In [0]:
threshold = 0.5
(log_reg.predict_proba(X_test_imputed)[:, 1] > threshold).astype(int)

In [0]:
manual_predictions = (log_reg.predict_proba(X_test_imputed)[:,1] > threshold).astype(int)
direct_predictions = log_reg.predict(X_test_imputed)

all(manual_predictions == direct_predictions)

### How accurate is the Logistic Regression?

In [0]:
score = log_reg.score(X_train_imputed, y_train)
print('Train Accuracy Score', score)

In [0]:
score = log_reg.score(X_test_imputed, y_test)
print('Test Accuracy Score', score)

In [0]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg, X_train_imputed, y_train, cv=10)
print('Cross-Validation Accuracy Scores', scores)

In [0]:
scores = pd.Series(scores)
scores.min(), scores.mean(), scores.max()

In [0]:
X_train_imputed.shape

In [0]:
y_pred = log_reg.predict(X_train_imputed)

In [0]:
len(y_pred)

In [0]:
len(y_train)

In [0]:
y_pred[:5]

In [0]:
y_train[:5].values

In [0]:
correct_predictions = 4
total_predictions = 5
accuracy = correct_predictions / total_predictions
print(accuracy)

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train[:5], y_pred[:5])

In [0]:
y_train.value_counts(normalize=True)

### What's the math for the Logistic Regression?

https://en.wikipedia.org/wiki/Logistic_function

https://en.wikipedia.org/wiki/Logistic_regression#Probability_of_passing_an_exam_versus_hours_of_study

In [0]:
log_reg.coef_

In [0]:
log_reg.intercept_

In [0]:
# The logistic sigmoid "squishing" function, 
# implemented to work with numpy arrays

def sigmoid(x):
    return 1 / (1 + np.e**(-x))

In [0]:
sigmoid(np.dot(log_reg.coef_, test_case.T) + log_reg.intercept_)

In [0]:
sigmoid(log_reg.coef_ @ test_case.T + log_reg.intercept_)

In [0]:
log_reg.predict_proba(test_case)

## Feature Engineering

Get the [Category Encoder](http://contrib.scikit-learn.org/categorical-encoding/) library

If you're running on Google Colab:

```
!pip install category_encoders
```

If you're running locally with Anaconda:

```
!conda install -c conda-forge category_encoders
```

In [0]:
import seaborn as sns
sns_titanic = sns.load_dataset('titanic')
print(sns_titanic.shape)

In [0]:
train.shape

In [0]:
sns_titanic.head()

In [0]:
# Titanic
train.head()

In [0]:
def make_features(X):
    X = X.copy()
    X['adult_male'] = (X['Sex'] == 'male') & (X['Age'] >= 16)
    X['alone'] = (X['SibSp'] == 0) & (X['Parch'] == 0)
    X['last_name'] = X['Name'].str.split(',').str[0]
    X['title'] = X['Name'].str.split(',').str[1].str.split('.').str[0]
    return X

In [0]:
train = make_features(train)
test  = make_features(test)

In [0]:
train['adult_male'].value_counts()

In [0]:
train['alone'].value_counts()

In [0]:
train['title'].value_counts()

In [0]:
sns_titanic.head()

## Assignment: real-world classification

We're going to check out a larger dataset - the [FMA Free Music Archive data](https://github.com/mdeff/fma). It has a selection of CSVs with metadata and calculated audio features that you can load and try to use to classify genre of tracks. To get you started:

### Get and unzip the data

#### Google Colab

In [2]:
!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
!unzip fma_metadata.zip

--2019-05-06 21:32:40--  https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
Resolving os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)... 86.119.28.13, 2001:620:5ca1:2ff::ce53
Connecting to os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)|86.119.28.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358412441 (342M) [application/zip]
Saving to: ‘fma_metadata.zip.3’

fma_metadata.zip.3  100%[===================>] 341.81M  26.0MB/s    in 14s     

2019-05-06 21:32:54 (24.1 MB/s) - ‘fma_metadata.zip.3’ saved [358412441/358412441]

Archive:  fma_metadata.zip
replace fma_metadata/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 bunzipping: fma_metadata/README.txt  
replace fma_metadata/checksums? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 bunzipping: fma_metadata/checksums  
replace fma_metadata/not_found.pickle? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 bunzipping: fma_metadata/not_found.pickle  
replace fma_metadata/raw_genres.csv? [y]es, [n]o, [A]ll, [N]one,

#### Windows
- Download the [zip file](https://os.unil.cloud.switch.ch/fma/fma_metadata.zip)
- You may need to use [7zip](https://www.7-zip.org/download.html) to unzip it


#### Mac
- Download the [zip file](https://os.unil.cloud.switch.ch/fma/fma_metadata.zip)
- You may need to use [p7zip](https://superuser.com/a/626731) to unzip it

### Look at first 3 lines of raw file

In [3]:
!head -n 4 fma_metadata/tracks.csv

,album,album,album,album,album,album,album,album,album,album,album,album,album,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,set,set,track,track,track,track,track,track,track,track,track,track,track,track,track,track,track,track,track,track,track,track
,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,tags,title,tracks,type,active_year_begin,active_year_end,associated_labels,bio,comments,date_created,favorites,id,latitude,location,longitude,members,name,related_projects,tags,website,wikipedia_page,split,subset,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,genres_all,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
track_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,,4,1,<p></p>,6073,,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,,,"<p>A 

### Read with pandas
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [0]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.linear_model import LogisticRegression


tracks = pd.read_csv('fma_metadata/tracks.csv', header=[0,1], index_col=0)

In [5]:
tracks.head()

album                                                     \
         comments         date_created        date_released engineer   
track_id                                                               
2               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
3               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
5               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
10              0  2008-11-26 01:45:08  2008-02-06 00:00:00      NaN   
20              0  2008-11-26 01:45:05  2009-01-06 00:00:00      NaN   

                                                                          \
         favorites id                                information listens   
track_id                                                                   
2                4  1                                    <p></p>    6073   
3                4  1                                    <p></p>    6073   
5                4  1                                    <p></p>    6073   
10               4  6                                        NaN   47632   
20               2  4  <p> "spiritual songs" from Nicky Cook</p>    2710   

                        ...       track                         \
         producer tags  ... information interest language_code   
track_id                ...                                      
2             NaN   []  ...         NaN     4656            en   
3             NaN   []  ...         NaN     1470            en   
5             NaN   []  ...         NaN     1933            en   
10            NaN   []  ...         NaN    54881            en   
20            NaN   []  ...         NaN      978            en   

                                                                              \
                                                    license listens lyricist   
track_id                                                                       
2         Attribution-NonCommercial-ShareAlike 3.0 Inter...    1293      NaN   
3         Attribution-NonCommercial-ShareAlike 3.0 Inter...     514      NaN   
5         Attribution-NonCommercial-ShareAlike 3.0 Inter...    1151      NaN   
10        Attribution-NonCommercial-NoDerivatives (aka M...   50135      NaN   
20        Attribution-NonCommercial-NoDerivatives (aka M...     361      NaN   

                                                 
         number publisher tags            title  
track_id                                         
2             3       NaN   []             Food  
3             4       NaN   []     Electric Ave  
5             6       NaN   []       This World  
10            1       NaN   []          Freeway  
20            3       NaN   []  Spiritual Level  

[5 rows x 52 columns]

### Fit Logistic Regression!

In [6]:
tracks.columns.tolist()

[('album', 'comments'),
 ('album', 'date_created'),
 ('album', 'date_released'),
 ('album', 'engineer'),
 ('album', 'favorites'),
 ('album', 'id'),
 ('album', 'information'),
 ('album', 'listens'),
 ('album', 'producer'),
 ('album', 'tags'),
 ('album', 'title'),
 ('album', 'tracks'),
 ('album', 'type'),
 ('artist', 'active_year_begin'),
 ('artist', 'active_year_end'),
 ('artist', 'associated_labels'),
 ('artist', 'bio'),
 ('artist', 'comments'),
 ('artist', 'date_created'),
 ('artist', 'favorites'),
 ('artist', 'id'),
 ('artist', 'latitude'),
 ('artist', 'location'),
 ('artist', 'longitude'),
 ('artist', 'members'),
 ('artist', 'name'),
 ('artist', 'related_projects'),
 ('artist', 'tags'),
 ('artist', 'website'),
 ('artist', 'wikipedia_page'),
 ('set', 'split'),
 ('set', 'subset'),
 ('track', 'bit_rate'),
 ('track', 'comments'),
 ('track', 'composer'),
 ('track', 'date_created'),
 ('track', 'date_recorded'),
 ('track', 'duration'),
 ('track', 'favorites'),
 ('track', 'genre_top'),
 ('t

In [7]:
tracks.isnull().sum()

album   comments                  0
        date_created           3529
        date_released         36280
        engineer              91279
        favorites                 0
        id                        0
        information           23425
        listens                   0
        producer              88514
        tags                      0
        title                  1025
        tracks                    0
        type                   6508
artist  active_year_begin     83863
        active_year_end      101199
        associated_labels     92303
        bio                   35418
        comments                  0
        date_created            856
        favorites                 0
        id                        0
        latitude              62030
        location              36364
        longitude             62030
        members               59725
        name                      0
        related_projects      93422
        tags                

In [0]:
trackstrack= tracks.track

In [9]:
trackstrack.isnull().sum()

bit_rate              0
comments              0
composer         102904
date_created          0
date_recorded    100415
duration              0
favorites             0
genre_top         56976
genres                0
genres_all            0
information      104225
interest              0
language_code     91550
license              87
listens               0
lyricist         106263
number                0
publisher        105311
tags                  0
title                 1
dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
features = ['duration', 'bit_rate']
target  = 'genres'

X = trackstrack[features]
y= trackstrack[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)





In [0]:

imputer = SimpleImputer()
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed  = imputer.transform(X_test)

In [12]:
log_reg = LogisticRegression(solver= 'lbfgs', multi_class='auto')
log_reg.fit(X_train_imputed, y_train)
print('Prediction :', log_reg.predict(X_test))
print('Predicted probabilities', log_reg.predict_proba(X_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Prediction : ['[21]' '[21]' '[21]' ... '[21]' '[21]' '[21]']
Predicted probabilities [[9.03814013e-04 1.22792768e-04 2.58403128e-04 ... 6.69225885e-04
  4.74501329e-04 3.84596514e-03]
 [1.34036307e-03 2.47375556e-05 1.09550039e-04 ... 7.34754909e-04
  3.69406053e-04 2.41641010e-02]
 [1.29528523e-03 4.62257414e-05 1.60102450e-04 ... 7.84173957e-04
  4.41669514e-04 1.44878368e-02]
 ...
 [1.34035670e-03 2.47375008e-05 1.09549766e-04 ... 7.34753779e-04
  3.69404918e-04 2.41661845e-02]
 [1.34011062e-03 2.47353815e-05 1.09539209e-04 ... 7.34710026e-04
  3.69361053e-04 2.42467431e-02]
 [9.03873851e-04 1.22793763e-04 2.58406862e-04 ... 6.69220935e-04
  4.74514485e-04 3.83821527e-03]]


In [14]:
trackstrack.genres

track_id
2                    [21]
3                    [21]
5                    [21]
10                   [10]
20              [76, 103]
26              [76, 103]
30              [76, 103]
46              [76, 103]
48              [76, 103]
134                  [21]
135              [45, 58]
136              [45, 58]
137               [1, 32]
138               [1, 32]
139                  [17]
140                  [17]
141                  [17]
142                  [17]
144                   [4]
145                   [4]
146                   [4]
147                   [4]
148                   [1]
149                   [1]
150                   [1]
151                  [25]
152                  [25]
153                  [26]
154                  [26]
155                  [26]
               ...       
155290    [18, 107, 1235]
155291    [18, 107, 1235]
155292    [18, 107, 1235]
155293    [18, 107, 1235]
155294    [18, 107, 1235]
155295    [18, 107, 1235]
155296    [18, 107, 1235]
155

This dataset is bigger than many you've worked with so far, and while it should fit in Colab, it can take awhile to run. That's part of the challenge!

Your tasks:
- Clean up the variable names in the dataframe
- Use logistic regression to fit a model predicting (primary/top) genre
- Inspect, iterate, and improve your model
- Answer the following questions (written, ~paragraph each):
  - What are the best predictors of genre?
  - What information isn't very useful for predicting genre?
  - What surprised you the most about your results?

*Important caveats*:
- This is going to be difficult data to work with - don't let the perfect be the enemy of the good!
- Be creative in cleaning it up - if the best way you know how to do it is download it locally and edit as a spreadsheet, that's OK!
- If the data size becomes problematic, consider sampling/subsetting, or [downcasting numeric datatypes](https://www.dataquest.io/blog/pandas-big-data/).
- You do not need perfect or complete results - just something plausible that runs, and that supports the reasoning in your written answers

If you find that fitting a model to classify *all* genres isn't very good, it's totally OK to limit to the most frequent genres, or perhaps trying to combine or cluster genres as a preprocessing step. Even then, there will be limits to how good a model can be with just this metadata - if you really want to train an effective genre classifier, you'll have to involve the other data (see stretch goals).

This is real data - there is no "one correct answer", so you can take this in a variety of directions. Just make sure to support your findings, and feel free to share them as well! This is meant to be practice for dealing with other "messy" data, a common task in data science.

## Resources and stretch goals

- Check out the other .csv files from the FMA dataset, and see if you can join them or otherwise fit interesting models with them
- [Logistic regression from scratch in numpy](https://blog.goodaudience.com/logistic-regression-from-scratch-in-numpy-5841c09e425f) - if you want to dig in a bit more to both the code and math (also takes a gradient descent approach, introducing the logistic loss function)
- Create a visualization to show predictions of your model - ideally show a confidence interval based on error!
- Check out and compare classification models from scikit-learn, such as [SVM](https://scikit-learn.org/stable/modules/svm.html#classification), [decision trees](https://scikit-learn.org/stable/modules/tree.html#classification), and [naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html). The underlying math will vary significantly, but the API (how you write the code) and interpretation will actually be fairly similar.
- Sign up for [Kaggle](https://kaggle.com), and find a competition to try logistic regression with
- (Not logistic regression related) If you enjoyed the assignment, you may want to read up on [music informatics](https://en.wikipedia.org/wiki/Music_informatics), which is how those audio features were actually calculated. The FMA includes the actual raw audio, so (while this is more of a longterm project than a stretch goal, and won't fit in Colab) if you'd like you can check those out and see what sort of deeper analysis you can do.